# SQLite

In [1]:
import pandas as pd
import sqlite3 as sql

#since we are looking at poems, we'll bump up the width of our columns
pd.set_option("display.max_colwidth", 250)

### Creating a connection to the poetry_kids.db database
 - tell python what database to connect to (`sql.connect( )`)
 - create a cursor to allow passing queries directly to the database

In [2]:
conn = sql.connect('../data/poetry_kids.db')
cur = conn.cursor()

### Checking what tables exist within a database
- the cursor executes the SQL query passed to `.execute( )`
- the results of your query are loaded to the cursor
- the `fetchall()` method returns those results so you can store them in a variable

In [3]:
# Execute tells the cursor to run the query provided

cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")

# fetchall() allows us to see everything returned by our query, which we store in our 'available_tables' variable

available_tables = (cur.fetchall())

available_tables

[('author',),
 ('emotion',),
 ('gender',),
 ('grade',),
 ('poem',),
 ('poem_emotion',)]

Let's compare this to the **entity relationship diagram** (ERD) for this database.

![ERD](assets/PoetryKids_erd.png)

## Using Pandas to place results into a DataFrame

We can SQL queries and convert the result to a _pandas_ DataFrame by using the `read_sql` method.

When building SQL queries, it may be useful to write the query independently and save to variable. Multi-line queries can use triple quotes (''' ''') to keep it together as one string.

For example, if we want to find the very short poems, we can use the following code. 

In [4]:
query = '''
SELECT text
FROM poem
WHERE char_count < 20;
'''

pd.read_sql(query, conn)

,text
0,hello friend
1,bye
2,i like chicken
3,"star, star,"
4,lol life rules
5,ghosts use magic
6,i love u
7,dragons are cool
8,may you bee ears
9,the time is gone


Let's now practice writing queries. Remember the process we can use:

    a. decide what tables you need   
    b. (if you need to use more than one table) decide how to connect your tables   
    c. decide which columns to select  
    d. decide what (if any) filters you want to apply in your `WHERE` clause

First, let's find the text of all poems with the title "cats".

In [7]:
query = '''
SELECT text
FROM poem
WHERE title = 'cats';
'''

pd.read_sql(query, conn)

,text
0,i love cats when they make sounds like this meow meow cat are raelly cute when they meow
1,cats aer fun cats are gerat
2,"furry ones, black ones, stript ones too every cat is different and so are you"
3,i like cats look at those hats but never mind that i like cats cats cats cats
4,cute awesome treats special
5,i am very fury i climb trees when i get scard and i go meow
6,cats. . . cats. . . cats. . . you know them you love them cats. . . cats. . . cats. . . !
7,cats are cute cats are cute i llike cats what about you
8,cats are really cute they cause a lot of trouble who cares i dont mind
9,cats are meowing and laying under the oak playing with a mouse


Now, let's see find the emotion_id associated with these poems. Keep the text of the poem, the poem_id, the emotion_id, and the intensity_percent.

Sort by the poem_id as the same poem can appear multiple times.

In [8]:
query = '''
SELECT text, poem_id, emotion_id, intensity_percent
FROM poem
JOIN poem_emotion
ON poem_id = poem_emotion.poem_id
WHERE poem.title = 'cats'
ORDER BY poem_id;
'''

pd.read_sql(query, conn)

,text,poem_id,emotion_id,intensity_percent
0,cats aer fun cats are gerat,2,1,45
1,cats are cute cats are cute i llike cats what about you,2,1,45
2,cats are meowing and laying under the oak playing with a mouse,2,1,45
3,cats are really cute they cause a lot of trouble who cares i dont mind,2,1,45
4,cats can be fat. to me some are mean. or funny like a bunny.,2,1,45
...,...,...,...,...
896429,"furry ones, black ones, stript ones too every cat is different and so are you",32842,4,44
896430,i am very fury i climb trees when i get scard and i go meow,32842,4,44
896431,i like cats look at those hats but never mind that i like cats cats cats cats,32842,4,44
896432,i love cats when they make sounds like this meow meow cat are raelly cute when they meow,32842,4,44


Finally, let's modify the above query to show the emotion name instead of the emotion_id. This will require joining with another table.

In [9]:
query = '''
SELECT text, poem_id, name, intensity_percent
FROM poem
JOIN poem_emotion
ON poem_id = poem_emotion.poem_id
JOIN emotion
ON poem_emotion.emotion_id = emotion.id
WHERE poem.title = 'cats'
ORDER BY poem_id;
'''

pd.read_sql(query, conn)

,text,poem_id,name,intensity_percent
0,cats aer fun cats are gerat,2,Anger,45
1,cats are cute cats are cute i llike cats what about you,2,Anger,45
2,cats are meowing and laying under the oak playing with a mouse,2,Anger,45
3,cats are really cute they cause a lot of trouble who cares i dont mind,2,Anger,45
4,cats can be fat. to me some are mean. or funny like a bunny.,2,Anger,45
...,...,...,...,...
896429,"furry ones, black ones, stript ones too every cat is different and so are you",32842,Joy,44
896430,i am very fury i climb trees when i get scard and i go meow,32842,Joy,44
896431,i like cats look at those hats but never mind that i like cats cats cats cats,32842,Joy,44
896432,i love cats when they make sounds like this meow meow cat are raelly cute when they meow,32842,Joy,44


## Best Practice - Close connections after use to prevent the tables from being locked

In [10]:
cur.close()
conn.close()